In [25]:
import glob
import os
import re
import pandas as pd
import numpy as np
from collections import defaultdict

## Extract mean score

In [112]:
# if not os.path.isdir('./results/extracted'):
#     os.mkdir('./results/extracted')

In [113]:
# p = re.compile('0[.]\d{4}\s\D{2}0[.]\d{2}\D') #[0.숫자4개 +띄어쓰기 + 문자 2개 0.숫자 2개 + 문자 1개
# data_paths = glob.glob('/home/aiffel0042/github_project/AVIDNet/results/*.txt')
# data_paths

[]

In [8]:
# for ori_data_path, data_path in zip(ori_data_paths, data_paths): # 데이터 경로에서 txt파일을 하나씩 불러온다.
#     with open(ori_data_path,'r') as f: 
#         all_data = f.read() # 전체 txt line을 읽어온다.
#         all_data = all_data.split('\n') # 띄어쓰기 기준으로 나눔
#         metric_line = all_data[-6:-2] # hard coding.. 맨 뒤 mean result 부분
#         extract = p.findall(str(metric_line))# 정규표현식에 맞는것만 뽑아내서 extract에 저장
#     with open(data_path,'a') as f:
#         for extract_line in extract:
#             f.write(str(extract_line)+'\n')

## Extract mean best scores among all random seeds

In [114]:
# input_dir = './results/lb25_mu1'
input_dir = './results/lb50_mu1'
out_dir = './results/extracted_for_best'
out_fname = 'lb50-mu1-b8,r0,1,2,3.csv'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [115]:
ori_data_paths = glob.glob(f'{input_dir}/*.txt')
fix_cols = ['th0.95-lb1.0', 'th0.95-lb0.75', 'th0.95-lb0.5',
            'th0.98-lb1.0', 'th0.98-lb0.75', 'th0.98-lb0.5']

# Extract paths for each colums such as baseline, fix-th0.95-lb1.0 and so on
data_paths_dict = DefaultOrderedDict(list)
for ori_data_path in ori_data_paths:
    # Exclude the case of the random seed 4 because of performance
    if '-r4-' not in ori_data_path:
        # Baseline
        if 'baseline' in ori_data_path:
            data_paths_dict['baseline'].append(ori_data_path)
        # FixMatch
        else:
            for fix_col in fix_cols:
                if fix_col in ori_data_path:
                    data_paths_dict[f'fix-{fix_col}'].append(ori_data_path)
                    break

data_paths_dict['baseline'][:3]

['./results/lb50_mu1/baseline-nl50-m1-b8-r0-th0.95-lb1.0-sc:step.txt',
 './results/lb50_mu1/baseline-nl50-m1-b8-r1-th0.95-lb1.0-sc:step.txt',
 './results/lb50_mu1/baseline-nl50-m1-b8-r2-th0.95-lb1.0-sc:step.txt']

In [116]:
p = re.compile('0[.]\d{4}') #[0.숫자4개]

cols = ('acc-all', 'ppv-all', 'ppv-covid', 'ppv-pneumonia', 'ppv-normal',
        'recall-all', 'recall-covid', 'recall-pneumonia', 'recall-normal',
        'f1-all', 'f1-covid', 'f1-pneumonia', 'f1-normal')
metrics_dfs = {}
for col, data_paths in data_paths_dict.items(): # 데이터 경로에서 txt파일을 하나씩 불러온다.
    metrics_dfs[col] = pd.DataFrame([], columns=cols)
    for ori_data_path in data_paths:
        with open(ori_data_path, 'r') as f: 
            all_data = f.read() # 전체 txt line을 읽어온다.
            all_data = all_data.split('\n') # 띄어쓰기 기준으로 나눔
            if 'baseline' in ori_data_path:
                candidates = ((327, 332), (662, 667), (997, 1002),
                              (1332, 1337), (1667, 1672))
            else:
                candidates = ((347, 352), (702, 707), (1057, 1062),
                              (1412, 1417), (1767, 1772))
            metrics = []
            for s_idx, e_idx in candidates:
                metric_line = all_data[s_idx:e_idx] # hard coding.. best result 부분
                metrics.append(list(map(float, p.findall(str(metric_line)))))# 정규표현식에 맞는것만 뽑기

            metrics_dfs[col] = pd.concat((metrics_dfs[col],
                                         pd.DataFrame(metrics, columns=cols))).reset_index(drop=True)

fm_cols = ['baseline'] + [f'fix-{c}' for c in fix_cols]
final_metrics_df = pd.DataFrame([], columns=cols)
for fm_col in fm_cols:
    final_metrics = []
    for col in cols:
        best_metrics = metrics_dfs[fm_col][col].to_numpy()
        mean, std = best_metrics.mean(), best_metrics.std()
        final_metrics.append(f'{mean:.4f} (±{std:.2f})')
        
    final_metrics_df = final_metrics_df.append(pd.DataFrame([final_metrics], columns=cols, index=(fm_col,)))

final_metrics_df

,acc-all,ppv-all,ppv-covid,ppv-pneumonia,ppv-normal,recall-all,recall-covid,recall-pneumonia,recall-normal,f1-all,f1-covid,f1-pneumonia,f1-normal
baseline,0.8247 (±0.02),0.8403 (±0.02),0.8703 (±0.04),0.7988 (±0.04),0.8516 (±0.03),0.8351 (±0.02),0.8415 (±0.05),0.8795 (±0.03),0.7844 (±0.04),0.8354 (±0.02),0.8542 (±0.03),0.8359 (±0.02),0.8160 (±0.03)
fix-th0.95-lb1.0,0.8175 (±0.02),0.8346 (±0.02),0.8770 (±0.04),0.7932 (±0.04),0.8336 (±0.04),0.8282 (±0.02),0.8025 (±0.07),0.8770 (±0.03),0.8052 (±0.05),0.8281 (±0.02),0.8352 (±0.03),0.8320 (±0.02),0.8170 (±0.02)
fix-th0.95-lb0.75,0.8195 (±0.02),0.8381 (±0.02),0.8899 (±0.05),0.7867 (±0.05),0.8378 (±0.03),0.8302 (±0.02),0.8020 (±0.07),0.8870 (±0.03),0.8016 (±0.05),0.8301 (±0.02),0.8404 (±0.04),0.8324 (±0.02),0.8175 (±0.02)
fix-th0.95-lb0.5,0.8217 (±0.02),0.8394 (±0.02),0.8865 (±0.04),0.7866 (±0.05),0.8451 (±0.03),0.8323 (±0.02),0.8115 (±0.06),0.8860 (±0.03),0.7995 (±0.05),0.8325 (±0.02),0.8451 (±0.03),0.8321 (±0.02),0.8203 (±0.02)
fix-th0.98-lb1.0,0.8258 (±0.02),0.8417 (±0.01),0.8776 (±0.04),0.8083 (±0.04),0.8392 (±0.03),0.8367 (±0.02),0.8280 (±0.06),0.8700 (±0.03),0.8120 (±0.05),0.8365 (±0.02),0.8492 (±0.03),0.8368 (±0.02),0.8235 (±0.02)
fix-th0.98-lb0.75,0.8267 (±0.02),0.8421 (±0.02),0.8726 (±0.04),0.8048 (±0.04),0.8489 (±0.03),0.8373 (±0.02),0.8370 (±0.05),0.8760 (±0.03),0.7990 (±0.05),0.8375 (±0.02),0.8526 (±0.03),0.8378 (±0.02),0.8219 (±0.02)
fix-th0.98-lb0.5,0.8258 (±0.02),0.8421 (±0.02),0.8786 (±0.04),0.8021 (±0.05),0.8458 (±0.03),0.8365 (±0.02),0.8300 (±0.06),0.8780 (±0.03),0.8016 (±0.05),0.8365 (±0.02),0.8511 (±0.03),0.8370 (±0.02),0.8214 (±0.02)


In [117]:
final_metrics_df.to_csv(f'{out_dir}/{out_fname}')

In [39]:
from collections import OrderedDict, Callable

class DefaultOrderedDict(OrderedDict):
    # Source: http://stackoverflow.com/a/6190500/562769
    def __init__(self, default_factory=None, *a, **kw):
        if (default_factory is not None and
           not isinstance(default_factory, Callable)):
            raise TypeError('first argument must be callable')
        OrderedDict.__init__(self, *a, **kw)
        self.default_factory = default_factory

    def __getitem__(self, key):
        try:
            return OrderedDict.__getitem__(self, key)
        except KeyError:
            return self.__missing__(key)

    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        self[key] = value = self.default_factory()
        return value

    def __reduce__(self):
        if self.default_factory is None:
            args = tuple()
        else:
            args = self.default_factory,
        return type(self), args, None, None, self.items()

    def copy(self):
        return self.__copy__()

    def __copy__(self):
        return type(self)(self.default_factory, self)

    def __deepcopy__(self, memo):
        import copy
        return type(self)(self.default_factory,
                          copy.deepcopy(self.items()))

    def __repr__(self):
        return 'OrderedDefaultDict(%s, %s)' % (self.default_factory,
                                               OrderedDict.__repr__(self))

In [2]:
nl = 150
mu = 4
seed = 0

for nl in [25,50,100,150]:
    for seed in [0,1,2,3]:
        l1=[]
        l2=[]
        with open(f'./data/fixed_sample/nl{nl}/{mu-1}/train_ulb_{seed}.txt','r') as f:
            lines = f.readlines()
            for line in lines:
                l1.append(line)
        with open(f'./data/fixed_sample/nl{nl}/{mu}/train_ulb_{seed}.txt','r') as f:
            lines = f.readlines()
            for line in lines:
                l2.append(line)
        print(f"nl={nl},seed={seed},len={len(set(l1+l2))}")

FileNotFoundError: [Errno 2] No such file or directory: './data/nl25/3/train_ulb_0.txt'